<a href="https://colab.research.google.com/github/rahuljungbahadur/hands_on_ml_book/blob/main/chp13_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os

In [8]:
os.listdir()

['.config', 'sample_data']

In [9]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving aclImdb_v1.tar.gz to aclImdb_v1.tar.gz


KeyboardInterrupt: ignored

In [ ]:
os.listdir()

['.config', 'aclImdb_v1.tar.gz', 'sample_data']

In [ ]:
import tarfile

# open file
file = tarfile.open('aclImdb_v1.tar.gz')
  
# print file names
print(file.getnames())
  
# extract files
file.extractall('./imdb_dataset')
  
# close file
file.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
os.listdir('imdb_dataset/aclImdb/train/')

['neg',
 'urls_pos.txt',
 'urls_neg.txt',
 'labeledBow.feat',
 'unsupBow.feat',
 'urls_unsup.txt',
 'unsup',
 'pos']

In [ ]:
trainSet_posFp = 'imdb_dataset/aclImdb/train/pos'
trainSet_negFp = 'imdb_dataset/aclImdb/train/neg'

In [ ]:
import tensorflow as tf

In [ ]:
help(tf.data.Dataset.list_files)

Help on function list_files in module tensorflow.python.data.ops.dataset_ops:

list_files(file_pattern, shuffle=None, seed=None, name=None)
    A dataset of all files matching one or more glob patterns.
    
    The `file_pattern` argument should be a small number of glob patterns.
    If your filenames have already been globbed, use
    `Dataset.from_tensor_slices(filenames)` instead, as re-globbing every
    filename with `list_files` may result in poor performance with remote
    storage systems.
    
    Note: The default behavior of this method is to return filenames in
    a non-deterministic random shuffled order. Pass a `seed` or `shuffle=False`
    to get results in a deterministic order.
    
    Example:
      If we had the following files on our filesystem:
    
        - /path/to/dir/a.txt
        - /path/to/dir/b.py
        - /path/to/dir/c.py
    
      If we pass "/path/to/dir/*.py" as the directory, the dataset
      would produce:
    
        - /path/to/dir/b.py
    

In [ ]:
trainSet_posFp_random = tf.data.Dataset.list_files(file_pattern='imdb_dataset/aclImdb/train/pos/*.txt')
trainSet_negFp_random = tf.data.Dataset.list_files(file_pattern='imdb_dataset/aclImdb/train/neg/*.txt')

In [ ]:
os.listdir('imdb_dataset/aclImdb')

['README', 'imdbEr.txt', 'valSet', 'testSet', 'test', 'imdb.vocab', 'train']

In [ ]:
# os.mkdir('./imdb_dataset/aclImdb/testSet/neg')
os.mkdir('./imdb_dataset/aclImdb/testSet/pos')
os.mkdir('./imdb_dataset/aclImdb/valSet/neg')
os.mkdir('./imdb_dataset/aclImdb/valSet/pos')

### Split the test set into a validation set (15,000) and a test set (10,000).

In [ ]:
import shutil

## test set
[shutil.copy2(f'imdb_dataset/aclImdb/test/neg/{fp}', 'imdb_dataset/aclImdb/testSet/neg') for fp in os.listdir('imdb_dataset/aclImdb/test/neg')[:5000]]
[shutil.copy2(f'imdb_dataset/aclImdb/test/pos/{fp}', 'imdb_dataset/aclImdb/testSet/pos') for fp in os.listdir('imdb_dataset/aclImdb/test/pos')[:5000]]

## Valid Set
[shutil.copy2(f'imdb_dataset/aclImdb/test/neg/{fp}', 'imdb_dataset/aclImdb/valSet/neg') for fp in os.listdir('imdb_dataset/aclImdb/test/neg')[5000:]]
[shutil.copy2(f'imdb_dataset/aclImdb/test/pos/{fp}', 'imdb_dataset/aclImdb/valSet/pos') for fp in os.listdir('imdb_dataset/aclImdb/test/pos')[5000:]]



In [ ]:
(len(os.listdir('imdb_dataset/aclImdb/valSet/pos/')), len(os.listdir('imdb_dataset/aclImdb/valSet/neg/')))

(7500, 7500)

In [ ]:
(len(os.listdir('imdb_dataset/aclImdb/testSet/pos/')), len(os.listdir('imdb_dataset/aclImdb/testSet/neg/')))

(5000, 5000)

#### Qns 2 Done

## Use tf.data to create an efficient dataset for each set.

In [ ]:
help(tf.data.TextLineDataset)

In [4]:
import tensorflow as tf
import os

In [6]:
os.listdir()

['.config', 'sample_data']

In [3]:
# trainSet_posFp_random = tf.data.Dataset.list_files(file_pattern='imdb_dataset/aclImdb/train/pos/*.txt')
# trainSet_negFp_random = tf.data.Dataset.list_files(file_pattern='imdb_dataset/aclImdb/train/neg/*.txt')

InvalidArgumentError: ignored

In [1]:
trainSet_pos = tf.data.TextLineDataset(trainSet_posFp_random, num_parallel_reads=5).batch(1)
trainSet_neg = tf.data.TextLineDataset(trainSet_negFp_random, num_parallel_reads=5).batch(1)

NameError: ignored

In [ ]:
testSet_pos = tf.data.TextLineDataset(tf.data.Dataset.list_files('imdb_dataset/aclImdb/testSet/pos/'), num_parallel_reads=5).batch(1)
testSet_neg = tf.data.TextLineDataset(tf.data.Dataset.list_files('imdb_dataset/aclImdb/testSet/neg/'), num_parallel_reads=5).batch(1)

In [ ]:
valSet_pos = tf.data.TextLineDataset(tf.data.Dataset.list_files('imdb_dataset/aclImdb/valSet/pos/'), num_parallel_reads=5).batch(1)
valSet_neg = tf.data.TextLineDataset(tf.data.Dataset.list_files('imdb_dataset/aclImdb/valSet/neg/'), num_parallel_reads=5).batch(1)

## Qns 4 Create a binary classification model, using a TextVectorization layer

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization, Dense, Input

In [ ]:
help(TextVectorization)

In [ ]:
model1 = Sequential()

model1.add(Input(shape=(1,), dtype=tf.string))




In [ ]:
tf_layer = TextVectorization(output_mode='int')

In [ ]:
tf_layer.adapt(trainSet_pos)

In [ ]:
model1.add(tf_layer)

In [ ]:
sample_file = []
with open(f'{trainSet_posFp}/7493_7.txt') as file:
  sample_file=file.read()

In [ ]:
sample_file.count('')

2831

In [ ]:
model1.predict([sample_file]).size

515

## Creating complete train dataset

In [179]:
trainSet = trainSet_pos.concatenate(trainSet_neg)

In [187]:
y_pos = tf.repeat(1, repeats=12500)
y_neg = tf.repeat(0, repeats=12500)
y_train = tf.concat([y_pos, y_neg], axis=0)

In [188]:
y_train

<tf.Tensor: shape=(25000,), dtype=int32, numpy=array([1, 1, 1, ..., 0, 0, 0], dtype=int32)>

In [191]:
testSet = testSet_pos.concatenate(testSet_neg)
y_test_pos = tf.repeat(1, repeats=5000)
y_test_neg = tf.repeat(0, repeats=5000)
y_test = tf.concat([y_test_pos, y_test_neg], axis=0)

In [192]:
valSet = valSet_pos.concatenate(valSet_neg)
y_val_pos = tf.repeat(1, repeats=7500)
y_val_neg = tf.repeat(0, repeats=7500)
y_val = tf.concat([y_val_pos, y_val_neg], axis=0)

In [ ]:
model1.add()